# Machine Learning Basics with Scikit-learn: Day 4

## Data pre-processing

One of the main tasks in machine learning is pre-processing the data. This means transforming the raw data in an efficient format. Some common issues that raw data have are:
* Presence of outliers
* Missing data
* Data with different scales
* Data without a normal distribution
* Redundant features
* Noisy features
* Data operationalization

The major steps in data pre-processing are:
1. **Data Cleaning**: Data can have many irrelevant and missing parts. To handle this part, data cleaning is done. It involves handling of missing data, noisy data etc. 
2. **Data Transformation**: Data and features' scales need to be changed in appropiate forms that respect the statistical principles. 
3. **Data Reduction**: Reducing the number of features and observations can increase the storage efficiency and reduce data storage and analysis costs. Also, prevent bias issues. 

The `sklearn` library provides several utility functions and transformer classes to change raw data into a representation that is more suitable for the downstream estimators.

Today, we will review three main practices that are core in data-preprocessing:
* Data standarization and normalization
* Feature selection
* Imputing data 

Let's start loading the libraries.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.datasets import load_boston
from sklearn.datasets import fetch_california_housing

## Datasets

For today's session, we will use the Boston houses' prices for demo purposes. You will use the California dataset for the exercises:

<table>
    <tr>
        <td style="text-align:center; width:50%">
            <h3>Boston house prices dataset</h3>
            <img src="https://www.bostonmagazine.com/wp-content/uploads/sites/2/2020/03/back-bay-brownstone.jpg" width="400px">
            <ul class="simple">
                <li><p>CRIM     per capita crime rate by town</p></li>
                <li><p>ZN       proportion of residential land zoned for lots over 25,000 sq.ft.</p></li>
                <li><p>INDUS    proportion of non-retail business acres per town</p></li>
                <li><p>CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)</p></li>
                <li><p>NOX      nitric oxides concentration (parts per 10 million)</p></li>
                <li><p>RM       average number of rooms per dwelling</p></li>
                <li><p>AGE      proportion of owner-occupied units built prior to 1940</p></li>
                <li><p>DIS      weighted distances to five Boston employment centres</p></li>
                <li><p>RAD      index of accessibility to radial highways</p></li>
                <li><p>TAX      full-value property-tax rate per \$10,000</p></li>
                <li><p>PTRATIO  pupil-teacher ratio by town</p></li>
                <li><p>B        1000(Bk - 0.63)^2 where Bk is the proportion of black people by town</p></li>
                <li><p>LSTAT    \% lower status of the population</p></li>
                <li><p>MEDV     Median value of owner-occupied homes in $1000’s</p></li>
            </ul>
        </td>
        <td style="text-align:center">
            <h3>California dataset </h3>
            <img src="https://cdn.vox-cdn.com/thumbor/xIC4EBWBo2QPsmB__soEniUe2ZU=/0x0:8736x5824/1820x1213/filters:focal(3670x2214:5066x3610):format(webp)/cdn.vox-cdn.com/uploads/chorus_image/image/56696503/shutterstock_154454792.0.0.jpeg" width="400px">
            <p>This dataset was derived from the 1990 U.S. census, using one row per census block group. A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people).</p>
            <ul class="simple">
                <li>MedInc median income in block</li>
                <li>HouseAge median house age in block</li>
                <li>AveRooms average number of rooms</li>
                <li>AveBedrms average number of bedrooms</li>
                <li>Population block population</li>
                <li>AveOccup average house occupancy</li>
                <li>Latitude house block latitude</li>
                <li>Longitude house block longitude</li>
            </ul>
        </td>
    </tr>
</table>

### Loading the Boston dataset

In [ ]:
boston_dataset = load_boston()

In [ ]:
boston_df = pd.DataFrame(boston_dataset.data, columns = boston_dataset.feature_names)
boston_df['PRICE'] = boston_dataset['target']
boston_df

### Exercise 1
Load the California dataset and create a DataFreme. This dataset contains 20,640 samples and 9 features.

Remember that the scikit-learn raw datasets come with the following objects:
* `data`
* `target`
* `feature_names`

In [ ]:
# Load the data from scikit-learn
california_dataset = fetch_california_housing()

In [ ]:
# Create the dataframe
#california_df = ....
#california_df['MedInc'] = ...
#california_df

## Data descriptive analysis
The first step is usually comprehend and analyze the data. This can be done by checking their descriptive statistics, distributions, and correlations.

We run the command `describe` to see the statistical information of this dataset: count, mean, standard deviation, minimum value, quartiles, and maximum value.

In [ ]:
round(boston_df.describe(),2)

We can also see each feature's distribtuion visually by calculating its **histogram.** This plot shows how many observations with a specific value `x` are present in the dataset. In statistical analysis, data should follow a normal distribution to be appropiaterly analyzed. 

Let's start checking the Boston dataset distribution. We will check first how the target (i.e., PRICE) is distributed.

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.hist(boston_df['PRICE'], edgecolor = 'black', bins = int(180/5))
plt.title("Histogram of Boston houses' prices")
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.show()

Now, let's see how the other variables are distributed

In [ ]:
boston_df.hist(figsize=(15,10))
plt.show()

Finally, we will check the correlations among the variables

In [ ]:
boston_df.corr()

### Excerise 2
Check the descriptive statistics and distributions of the California dataset

In [ ]:
# Calculate the descriptives

In [ ]:
# Plot the distribution of the target value ('MedInc')

In [ ]:
# Plot the distribution of all features

In [ ]:
# Calculate the correlations

## Standarization: Z-score

In general, learning algorithms benefit from standardization of the data set. If some outliers are present in the set, robust scalers or transformers are more appropriate. 

Standardization of datasets is a common requirement for many machine learning estimators implemented in scikit-learn. They might behave badly if the individual features do not more or less look like standard normally distributed data: **zero mean and unit variance**.

In practice, we often ignore the shape of the distribution and just **transform the data to center it by removing the mean value of each feature, then scale it by dividing non-constant features by their standard deviation**.

<img class="XqHOTb IGEbUc" alt="Z = \frac{x - \mu}{\sigma}" src="https://www.gstatic.com/education/formulas2/355397047/en/z_score.svg" role="img" data-atf="0" data-frt="0">

where *Z* is the standard score, *x* is the observed value, *u* is the mean of the sample, and is *s* the standard deviation of the sample. 

For instance, many elements used in the objective function of a learning algorithm assume that all features are centered around zero and have variance in the same order. If a feature has a variance that is orders of magnitude larger than others, it might dominate the objective function and make the estimator unable to learn from other features correctly as expected.

The preprocessing module provides the **StandardScaler** utility class, which is a quick and easy way to perform the following operation on an array-like dataset:

In [ ]:
# Load the scaler
scaler = preprocessing.StandardScaler()

In [ ]:
# Fit and transform the data
boston_df_scaled = scaler.fit_transform(boston_df)

In [ ]:
# Convert the data into a pandas Dataframe
boston_df_scaled = pd.DataFrame(boston_df_scaled, columns = boston_df.columns)

In [ ]:
# Get the descriptives. We round the results to 2 decimals. 
round(boston_df_scaled.describe(),2)

In [ ]:
# We plot the features' distribution.
boston_df_scaled.hist(figsize=(15,10))
plt.show()

### Exercise 3:
Use the `StandardScaler` to transform the California dataset. Then check that the means are equal to zero and standard deviation are equal to 1. 

In [ ]:
# Load the scaler
# scaler = ...

In [ ]:
# Fit and transform the data
# california_df_scaled = ...

In [ ]:
# Convert the data into a pandas Dataframe
# california_df_scaled = ...

In [ ]:
# Get the descriptives. We round the results to 2 decimals. 
# ....

In [ ]:
# We plot the features' distribution.
# ....

### Summary
* Z-score normalization handles outlier well but does not produce normalized data with the exact same scale.
* This method rescales data to have a mean equal to 0 and a standar deviation of 1. 

<img class="img-responsive img-thumbnail" src="https://www.simplypsychology.org/standardizing.svg" alt="Standard Normal Distribution (SND)">

## Min-max normalization: Scaling features to a range
An alternative standardization is scaling features to lie between a given minimum and maximum value, often between zero and one, or so that the maximum absolute value of each feature is scaled to unit size. This can be achieved using *MinMaxScaler* or *MaxAbsScaler*, respectively.

The formula is:
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/0222c9472478eec2857b8bcbfa4148ece4a11b84">

Here is an example to scale a toy data matrix to the [0, 1] range:

In [ ]:
# Load the scaler
min_max_scaler = preprocessing.MinMaxScaler()

In [ ]:
# Fit and transform the data
boston_df_scaled_minmax = min_max_scaler.fit_transform(boston_df)

In [ ]:
# Convert the data into a pandas Dataframe
boston_df_scaled_minmax = pd.DataFrame(boston_df_scaled_minmax, columns = boston_df.columns)

In [ ]:
# Get the descriptives. We round the results to 2 decimals. 
round(boston_df_scaled_minmax.describe(),2)

In [ ]:
boston_df_scaled_minmax.hist(figsize=(15,10))
plt.show()

### Exercise 4:
Use the `MinMaxScaler` to scale the California dataset. 

In [ ]:
# Load the scaler
#min_max_scaler = ...

In [ ]:
# Fit and transform the data
#california_df_scaled_minmax = ...

In [ ]:
# Convert the data into a pandas Dataframe
#california_df_scaled_minmax = ...

In [ ]:
# Get the descriptives. We round the results to 2 decimals. 
#...

In [ ]:
# We plot the features' distribution.
# ...

### Summary:
* Min-max normalization method guarantees all features will have the exact same scale but does not handle outliers well. 
* This transformation rescales the dataset to another range, preserving the original scale distances and standard deviation.

## Feature selection

Feature selection is the process of reducing the number of input variables when developing a predictive model. It is desirable to reduce the number of input variables to both reduce the computational cost of modeling, reducing the amount of data collected, and, in some cases, to improve the performance of the model.

For this exercise, we will split the dataset in two parts:
* the predictive features `X`
* the target `y`

In [ ]:
X_boston = boston_df.iloc[:,:-1]
y_boston = boston_df["PRICE"]

We check the data:

In [ ]:
X_boston

We do the same for the California datasets

In [ ]:
#X_california = california_df.iloc[:,1:]
#y_california = california_df["MedInc"]

In [ ]:
#X_california

### Removing features with low variance

VarianceThreshold is a simple baseline approach to feature selection. It removes all features whose variance doesn’t meet some threshold. By default, it removes all zero-variance features, i.e. features that have the same value in all samples.


In [ ]:
# Create the selector `VarianceThreshold`
selector = VarianceThreshold(threshold=(.8 * (1 - .8)))

In [ ]:
# Train the selector 
selector.fit(X_boston)

In [ ]:
# Create the dataframe with the features that explain at least 16% of the data variance
boston_df_selected_variance = X_boston[X_boston.columns[selector.get_support(indices=True)]]
boston_df_selected_variance

#### Exercise 5:
Create the selector `VarianceThreshold` with the California dataset.

In [ ]:
# Create the selector
# selector = ...

In [ ]:
# Train the selector using X_california
# ....

In [ ]:
# Create the dataframe using the california dataframe
#california_df_selected_variance = X_california[X_california.columns[selector.get_support(indices=True)]]
#california_df_selected_variance

### Univariate feature selection¶
Univariate feature selection works by selecting the best features based on univariate statistical tests. It can be seen as a preprocessing step to an estimator. Scikit-learn exposes feature selection routines as objects that implement the transform method.

We will check [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest), which removes all but the *k* highest scoring features.

How does this method select the *k* features? Depending on the type of problem (i.e., classification or regression), it runs F-tests to see what variables explain data's variance the most. These objects take as input a scoring function that returns univariate scores and p-values:

* For regression: `f_regression`, `mutual_info_regression`
* For classification: `chi2`, `f_classif`, `mutual_info_classif`

The methods based on F-test estimate the degree of linear dependency between two random variables. On the other hand, mutual information methods can capture any kind of statistical dependency, but being nonparametric, they require more samples for accurate estimation.

Since this dataset is usually tested with regressions, we will use the `f_regression` function to select the best *k* features in the model. This function tests the individual effect of each feature in a regression model. [More details here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression)

In [ ]:
# Create the SelectKBest object 
selector = SelectKBest(f_regression, k=2)

In [ ]:
# Fit the model
selector.fit(X_boston, y_boston)

We will see the F-scores of adding each feature to a regression model (which has `PRICE` as the target value)

In [ ]:
# Check the F-scores 
selector.scores_

And these are the p-values of these F-tests.

In [ ]:
# Check the p-values
selector.pvalues_

We select the features with the highest F-scores (smallest p-values). In this case, the 6th variable (RM) and the last variable (LSTAT) are the two features that explain the variance the most.

In [ ]:
# Create the dataframe with the two best features
boston_df_selected_kbest = X_boston[X_boston.columns[selector.get_support(indices=True)]]
boston_df_selected_kbest

#### Exercise 6
Create the selector `SelectKBest` with the California dataset (`X_california`)

In [ ]:
# Create the selector SelectKBest with the California data
#selector = ...

In [ ]:
# Fit the model. Remember to use `X_california` and `y_california`
#....

In [ ]:
# Create and print the selected variables
#california_df_selected_k_best = X_california[X_california.columns[selector.get_support(indices=True)]]
#california_df_selected_k_best

## Missing data

For various reasons, many real world datasets contain missing values, often encoded as blanks, NaNs or other placeholders. Such datasets however are incompatible with scikit-learn estimators which assume that all values in an array are numerical, and that all have and hold meaning. 

A basic strategy to use incomplete datasets is to discard entire rows and/or columns containing missing values. However, this comes at the price of losing data which may be valuable (even though incomplete). A better strategy is to **impute the missing values**, i.e., to infer them from the known part of the data. 

For this session, we will create an example where the Boston dataset will contain missing information (`NaN`)

In [ ]:
# Create the dataframe with missing data. It will remove 10% of the observations
X_boston_missing_data = X_boston.mask(np.random.random(X_boston.shape) < .1)
X_boston_missing_data

In [ ]:
# Check how many missing values each feature has. 
X_boston_missing_data.isna().sum()

We create the California dataset with missing data

In [ ]:
# Create the dataframe with missing data
#X_california_missing_data = X_california.mask(np.random.random(X_california.shape) < .1)
#X_california_missing_data

### Univariate feature imputation
One type of imputation algorithm is univariate, which imputes values in the i-th feature dimension using only non-missing values in that feature dimension.

The `SimpleImputer` class provides basic strategies for imputing missing values. Missing values can be imputed with a provided constant value, or using the statistics (mean, median or most frequent) of each column in which the missing values are located. This class also allows for different missing values encodings.

The strategies are:
* “mean”: replace missing values using the mean along each column. Can only be used with numeric data.
* “median”: replace missing values using the median along each column. Can only be used with numeric data.
* “most_frequent”: replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.
* “constant”: replace missing values with fill_value. Can be used with strings or numeric data.

The following snippet demonstrates how to replace missing values, encoded as np.nan, using the mean value of the columns (axis 0) that contain the missing values:

In [ ]:
# Create the SimpleImputer class
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
# Train and Transform the Missing data to Imputed data. 
X_boston_imputed = imp.fit_transform(X_boston_missing_data)

In [ ]:
# Create the dataframe
X_boston_imputed = pd.DataFrame(X_boston_imputed, columns = X_boston.columns)
X_boston_imputed

In [ ]:
# Check that the features do not have any missing data
X_boston_imputed.isna().sum()

#### Exercise 7: 
Create an imputer with the California dataset (`X_california`)

In [ ]:
# Create the SimpleImputer class
# imp = ...

In [ ]:
# Train and Transform the Missing data to Imputed data. Remember to use `X_california_missing_data`
# X_california_imputed = ....

In [ ]:
# Create the dataframe
#X_boston_imputed = pd.DataFrame(X_boston_imputed, columns = X_boston.columns)
#X_boston_imputed

### Multivariate feature imputation
Multivariate imputation algorithms use the entire set of available feature dimensions to estimate the missing values. Scikit-learn offers the `IterativeImputer` class, which models each feature with missing values as a function of other features, and uses that estimate for imputation. It does so in an iterated round-robin fashion: 
* At each step, a feature column is designated as output y and the other feature columns are treated as inputs X. A regressor is fit on (X, y) for known y. 
* Then, the regressor is used to predict the missing values of y.
* This is done for each feature in an iterative fashion, and then is repeated for max_iter imputation rounds. The results of the final imputation round are returned.

In [ ]:
# Create the IterativeImputer class
imp = IterativeImputer(max_iter=100, random_state=0)

In [ ]:
# Train and Transform the Missing data to Imputed data
X_boston_imputed = imp.fit_transform(X_boston_missing_data)

In [ ]:
# Create the dataframe
X_boston_imputed = pd.DataFrame(X_boston_imputed, columns = X_boston.columns)
X_boston_imputed

In [ ]:
# Check that there are no missing values
X_boston_imputed.isna().sum()

#### Exercise 8: 
Create an `IterativeImputer` with the California dataset (`X_california`)

In [ ]:
# Create the IterativeImputer class
#imp = ...

In [ ]:
# Train and transform the data using the imputer. Remember to use X_california_missing_data
#X_california_imputed = ...

In [ ]:
# Create the dataframe
#X_california_imputed = pd.DataFrame(X_california_imputed, columns = X_california.columns)
#X_california_imputed

### Nearest neighbors imputation

The `KNNImputer` class provides imputation for filling in missing values using the k-Nearest Neighbors approach. Each missing feature is imputed using values from `n_neighbors` nearest neighbors that have a value for the feature. The feature of the neighbors are averaged uniformly or weighted by distance to each neighbor. If a sample has more than one feature missing, then the neighbors for that sample can be different depending on the particular feature being imputed. 
Some rules:
* When the number of available neighbors is less than `n_neighbors` and there are no defined distances to the training set, the training set average for that feature is used during imputation. 
* If there is at least one neighbor with a defined distance, the weighted or unweighted average of the remaining neighbors will be used during imputation.
* If a feature is always missing in training, it is removed during transform. 

For more information, check the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html#sklearn.impute.KNNImputer)


In [ ]:
# Create the KNNImputer
imp = KNNImputer(n_neighbors=3, weights="uniform")

In [ ]:
# Train and Transform the Missing data to Imputed data
X_boston_imputed = imp.fit_transform(X_boston_missing_data)

In [ ]:
# Create the dataframe
X_boston_imputed = pd.DataFrame(X_boston_imputed, columns = X_boston.columns)
X_boston_imputed

In [ ]:
# We check that there are no NaN values
X_boston_imputed.isna().sum()

#### Exercise 9: 
Create an `KNNImputer` with the California dataset (`X_california`)

In [ ]:
# Create the KNNImputer class
# imp = ....

In [ ]:
# Train and transform the data using the KNNImputer. Remember to use X_california_missing_data
# X_california_imputed = ...

In [ ]:
# Create the dataframe
# X_california_imputed = pd.DataFrame(X_california_imputed, columns = X_california.columns)
# X_california_imputed